In [6]:
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import scipy.special as ss

In [2]:
class empty_class:
    pass

In [15]:
args = empty_class()
args.d = 15
args.k = 10
args.n_samples = 1000
args.seed = 42
args.gamma = 0.5

In [34]:
def get_samples(args):
    fname = os.path.join('samples', 
                         f'samples_{args.d}_{args.k}_{args.n_samples}_{args.seed}.pkl')
    if os.path.exists(fname):
        X = pickle.load(open(fname, 'rb'))
        return X
    else:
        q_k_d = ss.jacobi(args.k, (args.d-3)/2.0, (args.d-3)/2.0)
        legendre_k_d = q_k_d/q_k_d(1)
        X0 = torch.randn(args.n_samples,args.d)
        X0 = torch.nn.functional.normalize(X0, p=2, dim=1)
        acceptance_prob = torch.from_numpy(0.49 + 0.49*legendre_k_d(X0[:,args.d-1]))
        acceptance_vector = torch.bernoulli(acceptance_prob)
        print('Acc. prob. sum:', torch.norm(acceptance_prob, p=1), 'Acc. vec. sum:', torch.norm(acceptance_vector, p=1))
        accepted_rows = []
        for i in range(args.n_samples):
            if acceptance_vector[i] == 1:
                accepted_rows.append(i)
        accepted_rows_tensor = torch.tensor(accepted_rows).unsqueeze(1).expand([len(accepted_rows),args.d])
        X = torch.gather(X0, 0, accepted_rows_tensor)
        if not os.path.exists('samples'):
            os.makedirs('samples')
        pickle.dump(X, open(fname, 'wb'))
        return X

In [10]:
q_k_d = ss.jacobi(args.k, (args.d-3)/2.0, (args.d-3)/2.0)
legendre_k_d = q_k_d/q_k_d(1)

In [35]:
X = get_samples(args)

Acc. prob. sum: tensor(490.0014) Acc. vec. sum: tensor(501.)


In [37]:
X.shape

torch.Size([501, 15])